<a href="https://colab.research.google.com/github/dieko95/AlgoTrading/blob/develop/AlgoTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlgoTrading

## Download Alt Data (Tweets)

https://pypi.org/project/GetOldTweets3/



```
python Exporter.py --query @RedHat --since 2014-01-01 --until 2019-01-15 --maxtweets 5000000
```



In [0]:
# !pip install GetOldTweets3
# !pip install langdetect
# !pip install vaderSentiment
# !pip install -U -q PyDrive
# !pip install -q xlrd

 

In [0]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np 
from langdetect import detect
import time


In [6]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/open?id=1NSuc3X37Z2r29mxKt92JFs0krF63aNNM'
fluff, id = link.split('=')

print(id)


1NSuc3X37Z2r29mxKt92JFs0krF63aNNM


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
tweetsDf = pd.read_csv('Filename.csv')

In [8]:
tweetsDf.shape

(51603, 10)

##  Loading Prototype - snippet

![alt text](https://cdn-images-1.medium.com/max/800/1*G8yV2iaqqfaGfmRPRem2Fw.png)

In [22]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving aggDfDay_v0.1.csv to aggDfDay_v0.1.csv
User uploaded file "aggDfDay_v0.1.csv" with length 102638 bytes


In [0]:
# tweetsDf = pd.read_csv('redhatprototype.csv')
aggTweetsDay = pd.read_csv('aggDfDay_v0.1.csv')

## First Cleaning Iteration

In [0]:
# Erasing Short Tweets

start_time = time.time()

tweetsDf.text = tweetsDf.text.apply(lambda x: len(str(x).split()) < 3 and np.nan or x) #If the tweet is less than 7 characters create a NA place Holder 


# Erasing Misc Characters 

tweetsDf.text = tweetsDf.text.replace(r'@\w*', '', regex = True) # Erases all usernames (word preceding @)
tweetsDf.text = tweetsDf.text.replace(r'#\w*', '', regex = True) # Erases hashtags
tweetsDf.text = tweetsDf.text.replace(r'https://.+', '', regex = True) # Erases URLS

print("Middle of program--- %s seconds ---" % (time.time() - start_time))

tweetsDf.text = tweetsDf.text.replace(r'http://.+', '', regex = True) # Erases URLS
tweetsDf.text = tweetsDf.text.replace(r'=>', '', regex = True) # Erases =>
tweetsDf.text = tweetsDf.text.replace(r'[$-/:-?{-~"^_`\[\]]', '', regex = True) #Replace weird characters
tweetsDf.text = tweetsDf.text.apply(lambda x: len(str(x).split()) < 3 and np.nan or x) #If the tweet is less than 3 characters create a NA place Holder 



tweetsDf = tweetsDf.dropna(subset=['text'])


# Erasing Non English Tweets 

print("Before detect --- %s seconds ---" % (time.time() - start_time))


tweetsDf.text = tweetsDf.text.apply(lambda x: detect(str(x)) != 'en' and np.nan or x) # Creates NA place holder for text that is not in english 

tweetsDf = tweetsDf.dropna(subset=['text'])



print("--- %s seconds ---" % (time.time() - start_time))


Middle of program--- 0.25501370429992676 seconds ---
--- 276.7730143070221 seconds ---


In [0]:
# Tweets that were deleted - 110 for RHT 
print(tweetsDf.describe(), '\n','\n'
     'Tweets Without Info: ', tweetsDf.shape[0] - tweetsDf.text.dropna().shape[0])

tweetsDf = tweetsDf.dropna(subset=['text'])
tweetsDf.shape

           retweets     favorites
count  51603.000000  51603.000000
mean       2.851714      4.312811
std       18.691312     20.509410
min        0.000000      0.000000
25%        0.000000      0.000000
50%        0.000000      1.000000
75%        3.000000      4.000000
max     2877.000000   2330.000000 
 
Tweets Without Info:  110


In [0]:
start_time = time.time()

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()

tweet = []
number_favourites = []
retweets = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []
timeStamp = []


for i in range(tweetsDf.shape[0]):
    tweet.append(tweetsDf.iloc[i,4])
    number_favourites.append(tweetsDf.iloc[i,3])
    retweets.append(tweetsDf.iloc[i,2])
    timeStamp.append(tweetsDf.iloc[i,1])
    vs_compound.append(analyzer.polarity_scores(tweetsDf.iloc[i,4])['compound'])
    vs_pos.append(analyzer.polarity_scores(tweetsDf.iloc[i,4])['pos'])
    vs_neu.append(analyzer.polarity_scores(tweetsDf.iloc[i,4])['neu'])
    vs_neg.append(analyzer.polarity_scores(tweetsDf.iloc[i,4])['neg'])
    
    
print("--- %s seconds ---" % (time.time() - start_time))

In [0]:

twitterDf = DataFrame({'Tweet': tweet,
                        'Favourites': number_favourites,
                        'retweets' : retweets,
                        'timeStamp' : timeStamp,
                        'Compound': vs_compound,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})
twitterDf = twitterDf[['timeStamp','Tweet', 'Favourites', 'retweets','Compound',
                         'Positive', 'Neutral', 'Negative']]

twitterDf.shape

(44801, 8)

In [0]:
from google.colab import files

twitterDf.to_csv('cleanedTwitter_v0.1.csv')
files.download('cleanedTwitter_v0.1.csv')

## Aggregating By Day - Tweets

In [0]:
aggDf = twitterDf.drop(columns = 'Tweet')
aggDf.timeStamp = pd.to_datetime(aggDf.timeStamp) 
aggDf.set_index(aggDf["timeStamp"],inplace=True)
aggDfDay = aggDf.resample('D').sum()

# aggDfDay.Compound.value_counts()

aggDfDay

In [0]:
from google.colab import files

aggDfDay.to_csv('aggDfDay_v0.1.csv')
files.download('aggDfDay_v0.1.csv')

### Index for tweets loaded from Local System

In [66]:
# 
print(aggTweetsDay.shape,aggFundDfDay.shape)

aggTweetsDay = aggTweetsDay.iloc[1:1826,:]

print(aggTweetsDay.shape,aggFundDfDay.shape)

(1841, 7) (1825, 21)
(1825, 7) (1825, 21)


In [0]:
aggTweetsDay.timeStamp = pd.to_datetime(aggTweetsDay.timeStamp) 
aggTweetsDay.set_index(aggTweetsDay["timeStamp"],inplace=True)

## Loading Price and Fundamentals 


In [0]:

file_id = '1P-479AOJBUh1gAcTuxhGtsZHr_17nYV5'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('exported.xlsx')

In [76]:
import pandas as pd
fundData = pd.read_excel('exported.xlsx')
fundData.head()

,timeStamp,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,Adj Close t,Adj Close t+1,Return,Diluted EPS from Continuing Operations,EBIT Margin,EBITDA Margin,Revenue % Growth,EBITDA % Growth,EBIT % Growth,Net Income from Continuing Operations Sequential % Growth,Net Debt to Equity
0,2014-01-02,55.950001,56.000000,55.389999,55.830002,55.830002,1122500,NaN,NaN,55.830002,55.779999,-0.000896,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785
1,2014-01-03,55.900002,56.240002,55.450001,55.779999,55.779999,1103500,NaN,NaN,55.779999,56.340000,0.010039,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785
2,2014-01-06,56.180000,56.709999,55.930000,56.340000,56.340000,2028100,NaN,NaN,56.340000,57.419998,0.019169,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785
3,2014-01-07,56.610001,57.599998,56.400002,57.419998,57.419998,1834100,NaN,NaN,57.419998,56.990002,-0.007489,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785
4,2014-01-08,57.230000,57.389999,56.750000,56.990002,56.990002,1686600,NaN,NaN,56.990002,57.290001,0.005264,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785


### Aggregate by Day

In [77]:
print('Before Agg',aggFundDf.shape)
aggFundDf = fundData
aggFundDf.timeStamp = pd.to_datetime(aggFundDf.timeStamp) 
aggFundDf.set_index(aggFundDf["timeStamp"],inplace=True)
aggFundDfDay = aggFundDf.resample('D').sum()
print('After Agg',aggFundDf.shape)

Before Agg (1258, 22)
After Agg (1258, 20)


## Inner Join tables

In [78]:
finalDf = aggFundDfDay.join(aggTweetsDay,'timeStamp')
finalDf.head()

,Open,High,Low,Close,Adj Close,Volume,MA50,MA200,Adj Close t,Adj Close t+1,Return,Diluted EPS from Continuing Operations,EBIT Margin,EBITDA Margin,Revenue % Growth,EBITDA % Growth,EBIT % Growth,Net Income from Continuing Operations Sequential % Growth,Net Debt to Equity,timeStamp,Favourites,retweets,Compound,Positive,Neutral,Negative
timeStamp,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-01-02,55.950001,56.000000,55.389999,55.830002,55.830002,1122500,0.0,0.0,55.830002,55.779999,-0.000896,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785,2014-01-02,21.0,38.0,1.1823,1.203,3.797,0.000
2014-01-03,55.900002,56.240002,55.450001,55.779999,55.779999,1103500,0.0,0.0,55.779999,56.340000,0.010039,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785,2014-01-03,16.0,45.0,0.9725,0.624,5.233,0.143
2014-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2014-01-04,15.0,48.0,0.8392,0.628,3.372,0.000
2014-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2014-01-05,1.0,4.0,0.2732,0.231,2.769,0.000
2014-01-06,56.180000,56.709999,55.930000,56.340000,56.340000,2028100,0.0,0.0,56.340000,57.419998,0.019169,0.22,0.1451,0.1956,0.1713,0.1072,0.0437,0.2361,-0.5785,2014-01-06,3.0,9.0,1.4242,0.922,3.016,0.062
